# [문제]
- session id 부여

In [3]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()
    print(law[:50])

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 6. chain 구성
chain = prompt | llm

## 7. 대화 이력(히스토리) 관리 인스턴스 생성
history = InMemoryChatMessageHistory()

store = {}

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return history

## 9. 사용자 질문 받기
query = input('육아휴직과 관련된 질문하세요 >> ')

## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행
with_message_history.invoke(
    {'law': law, 'query': query},
    config={'configurable': {'session_id': 'abc'}}
)


남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 ( 약칭: 남녀고용평등법 )
[시행 20


In [5]:
get_history('abc')

InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직 기간은?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직 기간은 1년 이내로 할 수 있으며, 만약 다음과 같은 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다(남녀고용평등법 제19조(육아휴직) 제2항).\n\n- 같은 자녀를 대상으로 부모가 각각 3개월 이상 육아휴직을 사용한 경우\n- 「한부모가족지원법」 제4조제1호의 부 또는 모\n- 고용노동부령으로 정하는 장애아동의 부 또는 모\n\n즉, 기본적으로 1년이지만, 일부 경우에는 최대 1년 6개월까지 육아휴직을 사용할 수 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-1e942c8c-97a1-4f96-baaf-0990714311d9-0')])